In [6]:
%load_ext autoreload
%autoreload 2

import os
from utils import set_seed
import torch

set_seed(1234)
os.environ['HF_HOME'] = 'hf_cache' # Don't want model files in our home directory due to disk quota

torch.cuda.is_available()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [7]:
from transformers import LukeTokenizer
import const
from model import DocRedModel
import json

MODEL_NAME = const.LUKE_LARGE

rel2id = json.load(open('data/meta/rel2id.json'))
id2rel = {v: k for k, v in rel2id.items()}
tokenizer = LukeTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
model = DocRedModel(model_name=MODEL_NAME,
                    tokenizer=tokenizer,
                    num_class=len(rel2id)).to(const.DEVICE)

/data2/nhanse02/thesis/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from utils import read_docred
import json

train_samples = read_docred(fp='/data2/nhanse02/thesis/data/train_annotated.json', rel2id=rel2id, tokenizer=tokenizer)
dev_samples = read_docred(fp='/data2/nhanse02/thesis/data/dev.json', rel2id=rel2id, tokenizer=tokenizer)

/data2/nhanse02/thesis/data/dev.json: 100%|██████████| 998/998 [00:08<00:00, 121.84it/s]


In [9]:
from torch.utils.data import DataLoader
from utils import collate_fn

train_dataloader = DataLoader(train_samples, batch_size=4, shuffle=True, collate_fn=collate_fn, drop_last=True)
dev_dataloader = DataLoader(dev_samples, batch_size=8, shuffle=False, collate_fn=collate_fn, drop_last=False)

In [10]:
from train import train

train(model=model,
      train_dataloader=train_dataloader,
      dev_dataloader=dev_dataloader,
      dev_samples=dev_samples,
      id2rel=id2rel,
      num_epochs=30)

Validation: 100%|██████████| 125/125 [00:44<00:00,  2.82batch/s]


Epoch 1/30 F1: 0.5179 F1 Ign: 0.5013


Validation: 100%|██████████| 125/125 [00:46<00:00,  2.69batch/s]


Epoch 2/30 F1: 0.5106 F1 Ign: 0.4912


Validation: 100%|██████████| 125/125 [00:44<00:00,  2.80batch/s]


...

Epoch 28/30 F1: 0.6313 F1 Ign: 0.6126


Validation: 100%|██████████| 125/125 [00:42<00:00,  2.93batch/s]


Epoch 29/30 F1: 0.6329 F1 Ign: 0.6140


Validation: 100%|██████████| 125/125 [00:46<00:00,  2.71batch/s]


Epoch 30/30 F1: 0.6333 F1 Ign: 0.6149
